# NLP for Taxonomic Classification of DNA Sequences

This notebook demonstrates the application of natural language processing (NLP) techniques to classifying DNA sequences into different taxonomic groups. Although this notebook focuses on taxonomy, the approaches presented here can be extended to many other classification tasks in bioinformatics, such as classifying DNA into functional groups.

We'll explore two NLP approaches at different levels of complexity:

1. **Multinomial Naive Bayes Classifier:** This simple approach models the distribution of k-mers (subsequences of length k) for each taxon, without taking into account order or context. This allows us to predict the taxonomic group based on the presence and frequency of k-mers in the sequence.

2. **Fine-tuning a Transformer Model:** Transformer models, known for their ability to capture long-range relationships between sequence elements, dynamically weigh the importance of different elements based on their context. This allows them to identify important genetic elements even without explicit training. We'll leverage a 2.5 billion-parameter transformer model from InstaDeep, pre-trained on 850 genomes across phyla.

  For more information on the model, please see [Dalla-torre et al. 2023](https://www.biorxiv.org/content/10.1101/2023.01.11.523679v3). This section is adapted from [this notebook](https://github.com/huggingface/notebooks/blob/main/examples/nucleotide_transformer_dna_sequence_modelling_with_peft.ipynb) describing fine-tuning Nucleotide Transformers using Low-Rank Adaptation (LoRA), a technique allowing more efficient fine-tuning of large transformer models.

# Download Genomes from NCBI

First, we'll download and pre-process genomes from the National Center for Biotechnology Information (NCBI). Make sure to follow NCBI's [rules](https://www.ncbi.nlm.nih.gov/books/NBK25497/) regarding frequency and timing of requests and to provide them with your e-mail address in case of issues.

To download the genomic sequences, you'll first need to find the accession number associated with the genome. To do so, visit the [NCBI website](https://www.ncbi.nlm.nih.gov/) and search for your taxon of interest. You'll find the accession number listed near the top of the page for that database entry.


In [1]:
# @title Enter your e-mail, accession numbers, and names for each taxon:
email = "jacquelinekgrimm@gmail.com" # @param {type:"string"}
accession1 = "U00096" # @param {type:"string"}
taxon1 = "E. coli" # @param {type:"string"}
accession2 = "NC_000964" # @param {type:"string"}
taxon2 = "B. subtilis" # @param {type:"string"}

!pip install -q biopython tqdm

import pandas as pd
from Bio import Entrez, SeqIO
from tqdm import tqdm

def download_genome(accession, label, taxon_name, email):
    Entrez.email = email

    try:
        # Download the genome
        handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
        genome = SeqIO.read(handle, "fasta")
        handle.close()

        # Cut the genome into 500-nucleotide segments
        segment_size = 500
        segments = []
        for i in tqdm(range(0, len(genome.seq), segment_size), desc=f"Processing {taxon_name}"):
            segment = genome.seq[i:i+segment_size]
            if len(segment) == segment_size:
                segments.append(str(segment))

        # Create a dataframe with the DNA segments and their label
        df = pd.DataFrame({"sequence": segments, "label": [label] * len(segments)})
        return df

    except Exception as e:
        print(f"Error downloading genome {accession}: {str(e)}")
        return None

# Store the names of each taxon
taxon_names = {
    0: taxon1,
    1: taxon2
}

# Download and process each genome
df1 = download_genome(accession1, 0, taxon1, email)
df2 = download_genome(accession2, 1, taxon2, email)

if df1 is not None and df2 is not None:
    df = pd.concat([df1, df2], ignore_index=True)
    print()
    print(df)
else:
    print("One or more genomes failed to download. Please check the accession numbers and try again.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.7 MB/s eta 0:00:00


Processing B. subtilis: 100%|██████████| 8432/8432 [00:00<00:00, 481935.73it/s]


                                                sequence  label
0      AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATT...      0
1      TAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATT...      0
2      GTTGCGAGATTTGGACGGACGTTGACGGGGTCTATACCTGCGACCC...      0
3      CCGCTGGCAGTGACGGAACGGCTGGCCATTATCTCGGTGGTAGGTG...      0
4      TTCCAGCCAGGCAGTGGCGGATCAATATGCCGACTTCCTGCGCGAA...      0
...                                                  ...    ...
17709  ATAGAAATTCGTCCCTTATTACTTTAACTTATCCACATGTGAATAA...      1
17710  GTTTGTCCTCTTTTCCCAATCAACAATGCTGCCTTTTCACCTGTTA...      1
17711  GCCATTTGCGGATTTTGCTGCGCATTGCCAGCCATCATCAGTTTTT...      1
17712  ATGCGGACTATCTGCAGTGATCGGCTCTTTCACACTCGAGCATCCA...      1
17713  GATCAAGCGTATATAAGACAAACTGGCGGTTTGCAACTGATGTCCC...      1

[17714 rows x 2 columns]


*Optionally, save your dataset onto Google Drive by running the cells below.*

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Save the data as a CSV file
path = "/content/drive/MyDrive/DNA_Classification/data.csv" # Replace with your path
df.to_csv(path, index=False)
print(f"Data saved to {path}")

# Train a Multinomial Naive Bayes Classifier

Experiment with different k-mer lengths and values for alpha (the smoothing parameter) to optimize the accuracy of the classification model. This code block outputs the following metrics to help evaluate the classifier's performance:

**Cross-validation scores:** Cross-validation is a technique that helps assess how well the model performs on unseen data and reduces the risk of overfitting. In k-fold cross-validation, the training set is split into k equally sized subsets. The model is trained and evaluated k times, each time using a certain subset for validation and the remaining data for training. For each iteration, the accuracy is calculated on the validation set.

**Test set accuracy:** Before training, we'll reserve a portion of the data (in this case, 20%) as the test set. After training the classifier, we can evaluate its accuracy on the test set to assess its performance on unseen data.

**Confusion matrix:** The confusion matrix is a table summarizing the model's performance by comparing the predicted class labels to the actual class labels. The diagonal represents correctly classified instances. So in this case, the first row is the number of instances of taxon 1 *correctly* identified as taxon 1 followed by the number of instances of taxon 1 *incorrectly* classified as taxon 2. The second row is the number of instances of taxon 2 *incorrectly* classified as taxon 1 and so on.

In [2]:
# @title Enter k-mer length and alpha:
kmer_length = 6 # @param {type:"integer"}
alpha = 0.1 # @param {type:"number"}

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

# Function to create k-mers
def make_kmers(seq, kmer_length):
    return [seq[x:x + kmer_length].lower() for x in range(len(seq) - kmer_length + 1)]

# Create k-mers for each sequence in the dataframe
df['kmers'] = df['sequence'].apply(lambda x: make_kmers(x, kmer_length))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['kmers'], df['label'], test_size=0.2)

# Generate bag of words models
cv = CountVectorizer(analyzer=lambda x: x, lowercase=False)
X_train_bow = cv.fit_transform(X_train)
X_test_bow = cv.transform(X_test)

# Train the classifier and perform cross-validation
classifier = MultinomialNB(alpha=alpha)
cv_scores = cross_val_score(classifier, X_train_bow, y_train, cv=5)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean cross-validation score: {cv_scores.mean()}")

classifier.fit(X_train_bow, y_train)

# Predict classes of the test set
predictions = classifier.predict(X_test_bow)

# Print the classifier's accuracy on the test set
accuracy = accuracy_score(y_test, predictions)
print(f"Test set accuracy: {accuracy}")

# Print the confusion matrix
cm = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(cm)

Cross-validation scores: [0.92380952 0.92872265 0.9350741  0.93330981 0.9350741 ]
Mean cross-validation score: 0.9311980374365696
Test set accuracy: 0.925769122212814
Confusion Matrix:
[[1643  203]
 [  60 1637]]


## Classification Using the Trained Classifier

Enter a DNA sequence below to predict its class based on the trained Multinomial Naive Bayes classifier.

In [3]:
# @title Enter a DNA sequence:
sequence = "AAGAAGTAACCTTCGCTATTAAAACCAGTCAGTTGCTCTGGTTTGGTCAGCCGATTTTCAATAATGAAAC" # @param {type:"string"}

def is_valid_dna(sequence):
    valid_chars = set('ACGT')
    return all(char in valid_chars for char in sequence.upper())

# Predict the class of the sequence
def predict_sequence(sequence, classifier, cv, taxon_names, kmer_length=kmer_length):
    if not sequence:
        raise ValueError("Please enter a DNA sequence.")

    if not is_valid_dna(sequence):
        raise ValueError("Invalid DNA sequence. Only A, C, G, and T are allowed.")

    sequence_kmers = make_kmers(sequence.upper(), kmer_length)
    sequence_bow = cv.transform([sequence_kmers])
    predicted_class = classifier.predict(sequence_bow)
    predicted_proba = classifier.predict_proba(sequence_bow)
    predicted_taxon = taxon_names[predicted_class[0]]
    confidence_score = predicted_proba[0][predicted_class[0]]

    return predicted_taxon, confidence_score

# Predict the taxon for the input sequence
try:
    predicted_taxon, confidence_score = predict_sequence(sequence, classifier, cv, taxon_names)
    print(f"Predicted taxon: {predicted_taxon}")
    print(f"Confidence score: {confidence_score:.2f}")
except ValueError as e:
    print(f"Error: {str(e)}")

Predicted taxon: E. coli
Confidence score: 0.99


# Fine-tuning a Transformer Model

Transfer learning is a powerful approach that allows you to leverage large-scale pre-trained models to achieve state-of-the-art performance using relatively small amounts of task-specific data. In this section, we'll fine-tune a 2.5 billion-parameter pre-trained transformer model to classify these specific taxa. We'll utilize LoRA, a technique that introduces a small number of trainable parameters while keeping the majority of parameters frozen, making it possible to fine-tune large models efficiently.

We'll use a model from [Hugging Face](https://huggingface.co/), which provides access to a range of pre-trained models. The code below includes pushing your dataset and fine-tuned model to the platform. To do so, you'll first need to create an account. But you can still load and fine-tune the model without registration.

In [4]:
# @title Install required packages and log in to Hugging Face
!pip install -q transformers datasets huggingface_hub accelerate peft
!apt install -q git-lfs

from huggingface_hub import notebook_login
from datasets import Dataset, DatasetDict, load_dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, TrainingArguments, Trainer, AutoModelForSequenceClassification
import torch
from sklearn.metrics import matthews_corrcoef, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from peft import LoraConfig, TaskType, get_peft_model

# Log in to Hugging Face
notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━

In [5]:
# @title Push the dataset to Hugging Face
hf_username = "jacquelinegrimm" # @param {type:"string"}
dataset_name = "ecoli-bsubtilis" # @param {type:"string"}

# Function to create a custom dataset
def create_dataset(sequences, labels):
    dataset = Dataset.from_dict({"sequence": sequences, "label": labels})
    return dataset

# Extract sequences and labels from the dataframe
sequences = df["sequence"].tolist()
labels = df["label"].tolist()

# Create the training dataset
train_dataset = create_dataset(sequences, labels)

# Create DatasetDict
dataset = DatasetDict({
    "train": train_dataset
})

# Push to hub
hf_dataset_identifier = f"{hf_username}/{dataset_name}"
dataset.push_to_hub(hf_dataset_identifier)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jacquelinegrimm/ecoli-bsubtilis/commit/0ac61c1406ff7a5d7ee45b84692c39f48b33fe60', commit_message='Upload dataset', commit_description='', oid='0ac61c1406ff7a5d7ee45b84692c39f48b33fe60', pr_url=None, pr_revision=None, pr_num=None)

## Load the pretrained model and apply LoRA


In [6]:
# @title Enter the name of the pre-trained model:
model_name = "InstaDeepAI/nucleotide-transformer-2.5b-multi-species" # @param {type:"string"}

# Set the device to GPU
device = torch.device("cuda")

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Move the model to GPU
model = model.to(device)

# Configure the LoRA (Low-Rank Adaptation) settings
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, # Set the task type to sequence classification
    inference_mode=False, # Set inference mode to False for training
    r=1, # Set the rank of the update matrices
    lora_alpha=32, # Set the scaling factor for the update matrices
    lora_dropout=0.1,
    target_modules=["query", "value"],
)

# Apply LoRA to the pre-trained model
lora_classifier = get_peft_model(model, peft_config)
lora_classifier.print_trainable_parameters()

# Move the LoRA model to GPU
lora_classifier.to(device)

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/46.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/278M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-2.5b-multi-species and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 6,888,962 || all params: 2,544,177,285 || trainable%: 0.2707736619069767


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): EsmForSequenceClassification(
      (esm): EsmModel(
        (embeddings): EsmEmbeddings(
          (word_embeddings): Embedding(4105, 2560, padding_idx=1)
          (dropout): Dropout(p=0.0, inplace=False)
          (position_embeddings): Embedding(1002, 2560, padding_idx=1)
        )
        (encoder): EsmEncoder(
          (layer): ModuleList(
            (0-31): 32 x EsmLayer(
              (attention): EsmAttention(
                (self): EsmSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=2560, out_features=2560, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2560, out_features=1, bias=False)
                    )
                    (lora_B): ModuleDict(
   

## Tokenize datasets and fine-tune the model

Next, we'll need to tokenize the datasets, which converts the genomic sequences into a format that the model can understand. This involves breaking down the sequences into smaller units (tokens), allowing the model to process and learn from them efficiently. Then we'll fine-tune the LoRA-adapted model using the F1 score as the evaluation metric. The F1 score is used to evaluate the efficiency of binary classification models and considers both precision (the proportion of positive identifications that were correct) and recall (the proportion of actual positives that that were correctly identified).

In [7]:
# @title Tokenize datasets and fine-tune the model
new_model_name = "ecoli-bsubtilis" # @param {type:"string"}
num_train_epochs = 1 # @param {type:"integer"}
learning_rate = 5e-4 # @param {type:"number"}

# Load the train dataset from Hugging Face
train_dataset = load_dataset(hf_dataset_identifier, split="train")

# Get training data
train_sequences = train_dataset["sequence"]
train_labels = train_dataset["label"]

# Split the training dataset into training and validation datasets
train_sequences, validation_sequences, train_labels, validation_labels = train_test_split(
    train_sequences, train_labels, test_size=0.05, random_state=42)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create datasets from dictionaries
ds_train = Dataset.from_dict({"data": train_sequences, "labels": train_labels})
ds_validation = Dataset.from_dict({"data": validation_sequences, "labels": validation_labels})

# Function to tokenize the input sequences
def tokenize_function(examples):
    outputs = tokenizer(examples["data"])
    return outputs

# Create tokenized datasets
tokenized_datasets_train = ds_train.map(
    tokenize_function, batched=True, remove_columns=["data"])
tokenized_datasets_validation = ds_validation.map(
    tokenize_function, batched=True, remove_columns=["data"])

# Set training arguments
batch_size = 8
args_promoter = TrainingArguments(
    f"{new_model_name}-finetuned-lora-NucleotideTransformer",
    remove_unused_columns=False,
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=64,
    num_train_epochs=num_train_epochs,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1_score",
    label_names=["labels"],
    dataloader_drop_last=True,
    max_steps=1000
)

# Compute F1 score for binary classification
def compute_metrics_f1_score(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=-1)
    references = eval_pred.label_ids
    r = {"f1_score": f1_score(references, predictions)}
    return r

# Create the trainer
trainer = Trainer(
    lora_classifier.to(device),
    args_promoter,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_validation,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_f1_score,
)

# Fine-tune the model
train_results = trainer.train()

# Push the fine-tuned model to Hugging Face
kwargs = {
    "tags": ["DNA", "genomics"],
    "finetuned_from": model_name,
    "dataset": hf_dataset_identifier,
}
trainer.push_to_hub(**kwargs)

Generating train split:   0%|          | 0/17714 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

Map:   0%|          | 0/16828 [00:00<?, ? examples/s]

...

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

events.out.tfevents.1711545547.d39cc411bd0d.4103.0:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.6M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/jacquelinegrimm/ecoli-bsubtilis-finetuned-lora-NucleotideTransformer/commit/f08416fdc6ad18d2d7be8c1434b257f956353ee6', commit_message='End of training', commit_description='', oid='f08416fdc6ad18d2d7be8c1434b257f956353ee6', pr_url=None, pr_revision=None, pr_num=None)

## Classification Using the Fine-Tuned Transformer Model

In [8]:
# @title Enter a DNA sequence:
sequence = "AAGAAGTAACCTTCGCTATTAAAACCAGTCAGTTGCTCTGGTTTGGTCAGCCGATTTTCAATAATGAAAC" # @param {type:"string"}

def is_valid_dna(sequence):
    valid_chars = set('ACGT')
    return all(char in valid_chars for char in sequence.upper())

# Predict the class of the sequence
def predict_sequence(sequence, model, tokenizer):
    if not sequence:
        raise ValueError("Please enter a DNA sequence.")
    if not is_valid_dna(sequence):
        raise ValueError("Invalid DNA sequence. Only A, C, G, and T are allowed.")

    # Tokenize the input sequence
    inputs = tokenizer(sequence, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get the model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        predicted_proba = torch.softmax(logits, dim=1).squeeze().tolist()

    # Map the predicted class to the taxon name
    predicted_taxon = taxon_names[predicted_class]
    confidence_score = predicted_proba[predicted_class]

    return predicted_taxon, confidence_score

# Predict the taxon for the input sequence
try:
    predicted_taxon, confidence_score = predict_sequence(sequence, lora_classifier, tokenizer)
    print(f"Predicted taxon: {predicted_taxon}")
    print(f"Confidence score: {confidence_score:.2f}")
except ValueError as e:
    print(f"Error: {str(e)}")

Predicted taxon: E. coli
Confidence score: 0.95
